In [ ]:
# ===============================================================
# 📘 Comparative Study of PCA vs t-SNE on Image Data (CIFAR-10)
# Author: [Your Name]
# ===============================================================
# ⚙️ NOTE:
# Before running, go to Runtime → Change runtime type → select "GPU".
# This speeds up CNN feature extraction 10× faster than CPU.
# ===============================================================

# STEP 1 — Import libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.resnet50 import preprocess_input
import pandas as pd
import time

sns.set(style="whitegrid", palette="muted")
print("✅ Libraries imported successfully.")

# ===============================================================
# STEP 2 — Load CIFAR-10 dataset (replace this with otolith or fish images later)
# ===============================================================
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = y_train.flatten()

print(f"Training images: {X_train.shape}")
print(f"Unique labels: {np.unique(y_train)}")

# For faster computation, take only a subset
n_samples = 3000
X = X_train[:n_samples]
y = y_train[:n_samples]

# Display few sample images
fig, axes = plt.subplots(1, 5, figsize=(10, 2))
for i, ax in enumerate(axes):
    ax.imshow(X[i])
    ax.axis('off')
plt.suptitle("Sample CIFAR-10 Images", fontsize=14)
plt.show()

# ===============================================================
# STEP 3 — Preprocess images and extract CNN features using pretrained ResNet50
# ===============================================================
print("⏳ Extracting CNN features using ResNet50...")

IMG_SIZE = (224, 224)

# Resize & preprocess for ResNet
X_resized = tf.image.resize(X, IMG_SIZE)
X_preprocessed = preprocess_input(X_resized)

# Load pretrained ResNet50 WITHOUT top layers (we only need features)
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Extract features (each image → 2048-dim vector)
features = base_model.predict(X_preprocessed, batch_size=32, verbose=1)
print("✅ Features extracted:", features.shape)

# ===============================================================
# STEP 4 — Standardize features before PCA/t-SNE
# ===============================================================
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)
print("Example feature vector shape:", X_scaled[0].shape)

# ===============================================================
# STEP 5 — Apply PCA (2D)
# ===============================================================
print("⏳ Applying PCA (2D)...")
start_pca = time.time()

pca2 = PCA(n_components=2)
X_pca2 = pca2.fit_transform(X_scaled)

end_pca = time.time()
print(f"✅ PCA (2D) done in {end_pca-start_pca:.2f} seconds.")
print("Explained variance ratio (2D):", np.sum(pca2.explained_variance_ratio_))

# ===============================================================
# STEP 6 — Apply t-SNE (2D)
# ===============================================================
print("⏳ Applying t-SNE (2D, may take a few minutes)...")
start_tsne = time.time()

# Reduce to 50D first for speed
pca50 = PCA(n_components=50)
X_pca50 = pca50.fit_transform(X_scaled)

tsne2 = TSNE(n_components=2, perplexity=30, learning_rate=200, max_iter=1000, init='pca', random_state=42)
X_tsne2 = tsne2.fit_transform(X_pca50)

end_tsne = time.time()
print(f"✅ t-SNE (2D) done in {end_tsne-start_tsne:.2f} seconds.")

# ===============================================================
# STEP 7 — 2D Visualization
# ===============================================================
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

scatter1 = axes[0].scatter(X_pca2[:, 0], X_pca2[:, 1], c=y, s=10, cmap='tab10')
axes[0].set_title("PCA 2D Projection")
axes[0].legend(*scatter1.legend_elements(), title="Classes")

scatter2 = axes[1].scatter(X_tsne2[:, 0], X_tsne2[:, 1], c=y, s=10, cmap='tab10')
axes[1].set_title("t-SNE 2D Projection")
axes[1].legend(*scatter2.legend_elements(), title="Classes")

plt.suptitle("Comparison: PCA vs t-SNE on Image Features (2D)", fontsize=16)
plt.show()

# ===============================================================
# STEP 8 — Quantitative comparison (Silhouette score)
# ===============================================================
print("⏳ Calculating Silhouette Scores...")
sil_pca = silhouette_score(X_pca2, y)
sil_tsne = silhouette_score(X_tsne2, y)
print(f"✅ Silhouette (PCA):  {sil_pca:.4f}")
print(f"✅ Silhouette (t-SNE): {sil_tsne:.4f}")

# ===============================================================
# STEP 9 — Summary table
# ===============================================================
summary = pd.DataFrame({
    "Method": ["PCA (2D)", "t-SNE (2D)"],
    "Runtime (sec)": [end_pca-start_pca, end_tsne-start_tsne],
    "Silhouette Score": [sil_pca, sil_tsne],
    "Explained Variance (if applicable)": [np.sum(pca2.explained_variance_ratio_), "N/A"]
})
print("\n📊 Summary:")
print(summary)

# ===============================================================
# STEP 10 — Save embeddings and plot (for report)
# ===============================================================
np.save("X_pca2.npy", X_pca2)
np.save("X_tsne2.npy", X_tsne2)
np.save("X_pca3.npy", X_pca3)
np.save("X_tsne3.npy", X_tsne3)
plt.savefig("PCA_vs_TSNE_2D_comparison.png", dpi=300)
print("✅ Embeddings and plots saved successfully.")
